In [ ]:
import tensorflow as tf
import numpy as np

with tf.Graph().as_default():

    def init_weights(shape):
        return tf.Variable(tf.random_normal(shape, stddev=0.01))

    import tensorflow as tf
    import numpy as np

    from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
    mnist = read_data_sets('./temp', one_hot=True)
    trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
    trX = trX.reshape(-1, 28, 28, 1)
    teX = teX.reshape(-1, 28, 28, 1)

    X = tf.placeholder('float', [None, 28, 28, 1])
    Y = tf.placeholder('float', [None, 10])

    p_keep_conv = tf.placeholder(tf.float32)
    p_keep_hidden = tf.placeholder(tf.float32)

    w = init_weights([3,3,1,32])
    w2 = init_weights([3,3,32,64])
    w3 = init_weights([3,3,64,128])
    w4 = init_weights([128 * 4 * 4, 625])
    w_o = init_weights([625, 10])

    l1a = tf.nn.relu(tf.nn.conv2d(X, w, [1, 1, 1, 1], 'SAME')) # ?x28x28x32
    l1 = tf.nn.max_pool(l1a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # ?x14x14x32
    l1 = tf.nn.dropout(l1, p_keep_conv)

    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, [1, 1, 1, 1], 'SAME')) # ?x14x14x64
    l2 = tf.nn.max_pool(l2a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # ?x7x7x64
    l2 = tf.nn.dropout(l2, p_keep_conv)

    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, [1, 1, 1, 1], 'SAME')) # ?x7x7x128
    l3 = tf.nn.max_pool(l3a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # ?x4x4x128

    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]]) # reshape to ?x2048
    l3 = tf.nn.dropout(l3, p_keep_conv)

    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)

    pyx = tf.matmul(l4, w_o)

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pyx, Y))
    train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
    predict_op = tf.argmax(pyx, 1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predict_op, tf.argmax(Y, 1)), tf.float32))

    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        print("total training_batch iteration", (len(trX) / 128))
        
        for i in range(100):
            training_batch = zip(range(0, len(trX), 128),
                                 range(128, len(trX), 128))

            for start, end in training_batch:
                sess.run(train_op, feed_dict={X: trX[start:end], 
                                               Y: trY[start:end], 
                                               p_keep_conv: 0.8, 
                                               p_keep_hidden: 0.5})

                if int(start/128) % 100 == 0:
                    print('cost on mini batch iteration', start/128, sess.run(cost, feed_dict={X: trX[start:end], 
                                                                                 Y: trY[start:end], 
                                                                                 p_keep_conv: 0.8, 
                                                                                 p_keep_hidden: 0.5}))

            test_indices = np.arange(len(teX))
            np.random.shuffle(test_indices)
            test_indices = test_indices[0:256]
            
            print('test set accuracy on iteration',i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                            sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                            Y: teY[test_indices], 
                                                            p_keep_conv: 1.0, 
                                                            p_keep_hidden: 1.0})))

Extracting ./temp/train-images-idx3-ubyte.gz
Extracting ./temp/train-labels-idx1-ubyte.gz
Extracting ./temp/t10k-images-idx3-ubyte.gz
Extracting ./temp/t10k-labels-idx1-ubyte.gz
total training_batch iteration 429.6875
cost on mini batch iteration 0.0 2.30258
cost on mini batch iteration 100.0 2.30258
cost on mini batch iteration 200.0 1.43858
cost on mini batch iteration 300.0 0.229611
cost on mini batch iteration 400.0 0.133627
test set accuracy on iteration 0 0.97265625
cost on mini batch iteration 0.0 0.140539
cost on mini batch iteration 100.0 0.10173
cost on mini batch iteration 200.0 0.0986569
cost on mini batch iteration 300.0 0.0595457
cost on mini batch iteration 400.0 0.0966973
test set accuracy on iteration 1 0.98828125
cost on mini batch iteration 0.0 0.0637764
cost on mini batch iteration 100.0 0.0477665
cost on mini batch iteration 200.0 0.087853
cost on mini batch iteration 300.0 0.0474895
cost on mini batch iteration 400.0 0.0272908
test set accuracy on iteration 2 0.99